In [1]:
import ipywidgets as widgets
import numpy as np

In [2]:
preset_values = {
    "Identity": {
        "filter": [[0,0,0,0,0],
                   [0,0,0,0,0],
                   [0,0,1,0,0],
                   [0,0,0,0,0],
                   [0,0,0,0,0]],
        "factor": 1,
        "colour": True,
        "absolute": False,
        "min": 0,
        "max": 255},
    "Sobel X": {
        "filter": [[0, 0, 0, 0, 0],
                   [0, 1, 0,-1, 0],
                   [0, 2, 0,-2, 0],
                   [0, 1, 0,-1, 0],
                   [0, 0, 0, 0, 0]],
        "factor": 1,
        "colour": False,
        "absolute": True,
        "min": 32,
        "max": 216
    },
    "Sobel Y": {
        "filter": [[0, 0, 0, 0, 0],
                   [0, 1, 2, 1, 0],
                   [0, 0, 0, 0, 0],
                   [0,-1,-2,-1, 0],
                   [0, 0, 0, 0, 0]],
        "factor": 1,
        "colour": False,
        "absolute": True,
        "min": 32,
        "max": 216
    },
    "Gaussian": {
        "filter": [[1, 4, 6, 4, 1],
                   [4,16,24,16, 4],
                   [6,24,36,24, 6],
                   [4,16,24,16, 4],
                   [1, 4, 6, 4, 1]],
        "factor": 256,
        "colour": True,
        "absolute": False,
        "min": 0,
        "max": 255
    },
    "Adaptive Threshold": {
        "filter": [[-1, -4, -6, -4, -1],
                   [-4,-16,-24,-16, -4],
                   [-6,-24,220,-24, -6],
                   [-4,-16,-24,-16, -4],
                   [-1, -4, -6, -4, -1]],
        "factor": 256,
        "colour": False,
        "absolute": False,
        "min": 0,
        "max": 1
    },
    "Sharpen": {
        "filter": [[-1,-1,-1,-1,-1],
                   [-1, 2, 2, 2,-1],
                   [-1, 2, 8, 2,-1],
                   [-1, 2, 2, 2,-1],
                   [-1,-1,-1,-1,-1]],
        "colour": True,
        "absolute": False,
        "factor": 8,
        "min": 0,
        "max": 255  
    },
    "Threshold": {
        "filter": [[0,0,0,0,0],
                   [0,0,0,0,0],
                   [0,0,1,0,0],
                   [0,0,0,0,0],
                   [0,0,0,0,0]],
        "factor": 1,
        "colour": False,
        "absolute": False,
        "min": 128,
        "max": 128
    }
}

In [3]:
from pynq import Overlay
from demo_drivers import *
ol = Overlay('./base_logo.bit', download=False)

In [4]:
filter2d = ol.filter_pipeline_0


_updating = False

def new_preset(preset_dict):
    global _updating
    _updating = True
    coeffs = preset_dict["filter"]
    factor = preset_dict["factor"]
    for i, ci in enumerate(coeffs):
        for j, cj in enumerate(ci):
            filter_matrix.children[i].children[j].value = cj / factor
    colour_sel.value = "Colour" if preset_dict['colour'] else "Greyscale"
    absolute_sel.value = preset_dict['absolute']
    thresh_min.value = preset_dict['min']
    thresh_max.value = preset_dict['max']
    _updating = False

def on_preset(change):
    value = presets.value
    if value in preset_values:
        new_preset(preset_values[value])

def update_coeff(change):
    global _updating
    o = change['owner']
    o.element[:] = o.value * 256
    if not _updating:
        presets.value = "Custom"

def update_colour(change):
    global _updating
    o = change['owner']
    filter2d.colour[:] = 1 if o.value == "Colour" else 0
    if not _updating:
        presets.value = "Custom"

def update_absolute(change):
    global _updating
    o = change['owner']
    filter2d.absolute[:] = o.value
    if not _updating:
        presets.value = "Custom"

def update_maxthreshold(change):
    global _updating
    o = change['owner']
    filter2d.max_thresh[:] = o.value
    if not _updating:
        presets.value = "Custom"
        
def update_minthreshold(change):
    global _updating
    o = change['owner']
    filter2d.min_thresh[:] = o.value
    if not _updating:
        presets.value = "Custom"
        
hboxes = []
for i in range(5):
    inputs = []
    for j in range(5):
        w = widgets.BoundedFloatText(value=0, min=-4, max=4, step=0.1, layout={"width": "60px"})
        w.observe(update_coeff)
        w.element = filter2d.coeffs[i:i+1,j:j+1]
        inputs.append(w)
    hboxes.append(widgets.HBox(inputs))
filter_matrix = widgets.VBox(hboxes, layout={"border":"solid 2px"})

colour_sel = widgets.RadioButtons(options=["Colour", "Greyscale"], description="Filter Mode")
colour_sel.observe(update_colour)
absolute_sel = widgets.Checkbox(description="Absolute Output")
absolute_sel.observe(update_absolute)
thresh_min = widgets.IntSlider(description="Threshold", min=0, max=255)
thresh_min.observe(update_minthreshold)
thresh_max = widgets.IntSlider(description=" ", min=0, max=255, value=255)
thresh_max.observe(update_maxthreshold)
config = widgets.VBox([colour_sel, absolute_sel, thresh_min, thresh_max])

presets = widgets.Select(
    options=["Custom", "Identity", "Sobel X", "Sobel Y", "Gaussian", "Sharpen", "Threshold", "Adaptive Threshold"],
    rows=8)

presets.observe(on_preset)
filter_control = widgets.HBox([filter_matrix, config, presets])

In [5]:
filter_control

![Filter Pipeline](FilterPipeline.png)

In [11]:
from pynq import PL

In [12]:
ol.is_loaded()

True

In [13]:
PL._bitfile_name

'/home/xilinx/base_logo.bit'